# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf

In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

Reading configuration from ../src/config/default.conf
INFO:tensorflow:Restoring parameters from ..\models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000
Reading configuration from ../src/config/default.conf


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

# you sit West and hold:
hand = '73.KJ83.AT2.T962'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['PAD_START', '1D', '1S']

# what's your bid?

In [4]:
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, -1, -1, 0.1, sampler, False)

bot_bid.bid(['PAD_START', '1D', '1S']).bid

'X'

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(['PAD_START', '1D', '1S', 'X', '2S', '3H', 'PASS'])
bid.bid

1: 4H   0.6589 Expected Score: 101  
2: PASS 0.3114 Expected Score: 44   


'4H'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': '4H',
  'insta_score': 0.6589298844337463,
  'expected_score': 101.5339689096782},
 {'call': 'PASS',
  'insta_score': 0.3114379048347473,
  'expected_score': 44.314525486562495}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['KQTx.9xx.8xxx.8x 9x.ATxx.KQ9x.AJx AJ8xx.Qx.Jx.KQxx xx.KJ8x.ATx.T9xx',
 'Jxx.Qx.Jxxx.AJxx Q8x.AT9x.K9xx.Kx AKT9x.xxx.Q8.Q8x xx.KJ8x.ATx.T9xx',
 'J8xx.Txxx.xx.KQx Qx.AQ9x.KQJ8xx.8 AKT9x.x.9x.AJxxx xx.KJ8x.ATx.T9xx',
 'KJx.9xxx.Kxx.Qxx Axx.AQTx.J9xx.Jx QT98x.x.Q8x.AK8x xx.KJ8x.ATx.T9xx',
 'K8xx.xx.J9xx.Q8x J9.AQTx.KQ8x.Jxx AQTxx.9xx.xx.AKx xx.KJ8x.ATx.T9xx',
 'QJ8.Qxx.Jxxx.8xx AK.A9xx.KQ98x.Jx T9xxxx.Tx.x.AKQx xx.KJ8x.ATx.T9xx',
 'A9x.Qxx.QJ9x.8xx KTx.A9xx.K8xx.KJ QJ8xx.Tx.xx.AQxx xx.KJ8x.ATx.T9xx',
 'ATx.Txxx.QJx.Qxx QJ.AQ9x.98xx.A8x K98xxx.x.Kxx.KJx xx.KJ8x.ATx.T9xx',
 'A98x.9xx.J9x.Jxx KJ.AQxx.KQxxxx.8 QTxxx.Tx.8.AKQxx xx.KJ8x.ATx.T9xx',
 'KJx.Qx.K8xx.xxxx Axx.Axxx.QJ9x.Q8 QT98x.T9x.xx.AKJ xx.KJ8x.ATx.T9xx']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', '1D', '1S', 'X', '2S', '3H', 'PASS'])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

98x.xx.8xx.AQ8xx QJx.AQxx.KQJ9x.x AKTxx.T9x.xx.KJx xx.KJ8x.ATx.T9xx
KTx.Txx.J8xx.AJx 98.AQxx.K9xx.Kxx AQJxxx.9x.Qx.Q8x xx.KJ8x.ATx.T9xx
QJT8x.9xx.Q8x.Jx A.AQxx.KJ9xx.K8x K9xxx.Tx.xx.AQxx xx.KJ8x.ATx.T9xx
8xxx.Txxx..KJxxx Tx.AQ9x.QJ98x.A8 AKQJ9.x.Kxxxx.Qx xx.KJ8x.ATx.T9xx
Qxxx.xxx.KJxx.xx KJ.QT9x.Q8xx.KJ8 AT98x.Ax.9x.AQxx xx.KJ8x.ATx.T9xx
Q8xx.9xx.Qxx.QJx T9x.AQxx.K9xx.A8 AKJx.Tx.J8x.Kxxx xx.KJ8x.ATx.T9xx
T98x.9xx.J9x.KQx Qx.ATxx.KQxxx.Jx AKJxx.Qx.8x.A8xx xx.KJ8x.ATx.T9xx
Q8x.QT9x.J9x.QJx AJx.Axx.KQ8xx.xx KT9xx.xx.xx.AK8x xx.KJ8x.ATx.T9xx
AT9.9xx.K9x.Qxxx K8x.ATx.QJxxx.Ax QJxxx.Qxx.8x.KJ8 xx.KJ8x.ATx.T9xx
Kxx.xxx.Qxxx.K8x 98.AQxx.KJ98x.Ax AQJTxx.T9.x.QJxx xx.KJ8x.ATx.T9xx
Jxx.x.KJ98.Qxxxx QT8x.Q9xx.Qxx.AK AK9x.ATxx.xxx.J8 xx.KJ8x.ATx.T9xx
A9x.9xx.Q98xx.Kx Qx.AQTx.Kxxx.Q8x KJT8xx.xx.J.AJxx xx.KJ8x.ATx.T9xx
98xx.Txxx.Kxx.QJ KT.AQ9x.Jxxx.K8x AQJxx.x.Q98.Axxx xx.KJ8x.ATx.T9xx
JTx.Qxx.Q9xx.Jxx A98.A9xx.KJ8xx.Q KQxxx.Tx.x.AK8xx xx.KJ8x.ATx.T9xx
JTx.xx.98.KQJ8xx Q9x.A9xx.KQxxx.A AK8xx.QTx.Jxx.

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['KJT4.Q3.A2.K9754',
 '852.K95.QT986.Q6',
 'A976.J742.54.AT3',
 'Q3.AT86.KJ73.J82']

In [10]:
bidder_bots = [BotBid([False, False], hand, models, -1, -1, 0.1, sampler, False) for hand in hands]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

1: 4S   0.3891 Expected Score: 161  
2: PASS 0.5305 Expected Score: 106  


['1C',
 'PASS',
 '1H',
 'PASS',
 '1S',
 'PASS',
 '3S',
 'PASS',
 '4S',
 'PASS',
 'PASS',
 'PASS']